In [2]:
import cv2
from scipy import ndimage

In [4]:
import numpy as np
import matplotlib.pyplot as plt

import pywt
import pywt.data

import sys
import scipy


import os
from pathlib import Path

In [6]:
init = 'C:\\Users\\james\\OneDrive\\Desktop\\VIT\\Sem III\\Image Processing\\Project\\Virus Dataset\\original-16bit - Copy'
final = 'C:\\Users\\james\\OneDrive\\Desktop\\VIT\\Sem III\\Image Processing\\Project\\Virus Preprocessed'


KD1='C:\\Users\\james\\OneDrive\\Desktop\\VIT\\Sem III\\Image Processing\\Project\\Virus Preprocessed\\KD1'
KD2='C:\\Users\\james\\OneDrive\\Desktop\\VIT\\Sem III\\Image Processing\\Project\\Virus Preprocessed\\KD2'
KD3='C:\\Users\\james\\OneDrive\\Desktop\\VIT\\Sem III\\Image Processing\\Project\\Virus Preprocessed\\KD3'
KD4='C:\\Users\\james\\OneDrive\\Desktop\\VIT\\Sem III\\Image Processing\\Project\\Virus Preprocessed\\KD4'
KD5='C:\\Users\\james\\OneDrive\\Desktop\\VIT\\Sem III\\Image Processing\\Project\\Virus Preprocessed\\KD5'

TEMP='C:\\Users\\james\\OneDrive\\Desktop\\VIT\\Sem III\\Image Processing\\Project\\Virus Preprocessed\\TEMP'

In [9]:
def esi(data):
# print(data.ndim) = 2.. hence a 2d image

# ------- Changes end here -------

    w=len(data)
    l1=len(data[0])
    hist,bins = np.histogram(data.ravel(),256,[0,256])
    #print("Histogram of the image:")
    #print (hist) #Histogram of the image calculate

    #Step2, calculate Xa
    num =0
    denum=0
    l=len(hist)
    for i in range(1,l+1) :
        num+= hist[i-1]*i
        denum+=hist[i-1]

    #Step3 Clipping threshold and calculate clipping hstogram
    exposure = (1.0/l)*(num/(1.0*denum))
    xa=int(round(l*(1-exposure)))
    #print("Exposure value:", exposure)
    #print("Threshold value:", xa)
    #print("l:",l)

    clipthreshold =0

    s=0
    for i in range(1,l+1):
       s+=hist[i-1]
    clipthreshold = (1.0/l)*s

    hist_c=[]

    for i in range(1, l+1):
        if(hist[i-1]>clipthreshold):
            hist_c.append(clipthreshold)
        else:
            hist_c.append(hist[i-1])

    # print("Clipped Histogram of the image:")
    # print (hist_c) #Clipped Histogram of the image
    # print("Rounded exp value:", int(round(xa)))
    #Step 4 Histogram Sub Division and Equalization
    undexp= []
    overexp= []
    underexp= hist[0:int(round(xa+1))]
    overexp= hist[int(round(xa+1)):l]

    nl=0
    nu=0

    for i in range (0,xa+1):
        nl+=hist_c[i]
    for i in range (xa+1, l):
        nu+=hist_c[i]

    pl=[]
    pu=[]
    for i in range(0,int(round(xa+1))):
        # print(hist_c[i],"and", nl)
        pl.append(hist_c[i]/nl)

    for i in range(int(round(xa+1)),l):
        pu.append(hist_c[i]/nu)
        # print(pu[i], ":::", i)


    #Get corresponding CDF
    cl =[]
    cu =[]

    cl.append(pl[0])
    #print("len",len(pl))
    for r in range(1,len(pl)):
        cl.append(pl[r]+cl[r-1])

    #print("Length:",len(cl))
    cu.append(pu[0])
    for r in range(1,(len(pu))):
        cu.append(pu[r]+cu[r-1])

    ESIHEoutput = [[0 for x in range(l1)] for y in range(w)]
    # print("row and col",w, "and",l1)

    # print("Data:",data[0]) #Check this output @Nikhil
    for r in range(0,w ):
        for s in range (0,l1):
            if data[r][s]<(xa+1):
                #print(data[r][s])
                f=xa*cl[data[r][s]]
                # print("r",r,"s",s)
                ESIHEoutput[r][s]=round(f)
                # print("ESIHE", r," n ", s, ":",ESIHEoutput[r][s])
            else:
                f=(xa+1)+(255-xa)*cu[(data[r][s]-(xa+1))]
                ESIHEoutput[r][s]=round(f)


    #print("\nRESULT:")

    #print("Output matrix:\n\n")
    return np.array(ESIHEoutput)

In [11]:
#KD5
classset=set([])
p = Path(init)
c=0
for x in p.iterdir():
    
    try:
        image16 = cv2.imread(str(x), cv2.IMREAD_GRAYSCALE)

        if(image16.shape!=(41,41)):
            print('yes')

        img8 = image16

        all_zeros = not np.any(img8)
        if(all_zeros):
            continue

        coeffs2 = pywt.dwt2(img8, 'db1')
        LL, (LH, HL, HH) = coeffs2

        width = 41
        height = 41
        dim = (width, height)

        # resize image
        LL = (cv2.resize(LL, dim, interpolation = cv2.INTER_AREA)).astype('uint8')
        LH = (cv2.resize(LH, dim, interpolation = cv2.INTER_AREA)).astype('uint8')
        HL = (cv2.resize(HL, dim, interpolation = cv2.INTER_AREA)).astype('uint8')
        HH = (cv2.resize(HH, dim, interpolation = cv2.INTER_AREA)).astype('uint8')

        gauss3 = (ndimage.gaussian_filter(img8, 1, 3, mode='wrap')).astype('uint8')
        gauss5 = (ndimage.gaussian_filter(img8, 1, 5, mode='wrap')).astype('uint8')
        gi = (ndimage.sobel(img8, 1, mode='wrap')).astype('uint8')

        esimg = esi(img8)


        w = str(x)
        y = os.path.normpath(KD5+ '//' + w[w.rfind('\\')+1: w.rfind('.')])

        # Writing Files

        cv2.imwrite(str(y)+'-LL.png',LL)
        cv2.imwrite(str(y)+'-LH.png',LH)
        cv2.imwrite(str(y)+'-HL.png',HL)
        cv2.imwrite(str(y)+'-HH.png',HH)
        cv2.imwrite(str(y)+'-g3.png',gauss3)
        cv2.imwrite(str(y)+'-g5.png',gauss5)
        cv2.imwrite(str(y)+'-gi.png',gi)
        cv2.imwrite(str(y)+'-esi.png',esimg)
        #print(gr) #  Run check

        c = c+1
    except:
        continue
        #break

print('Total FILES Processed : ', c, end=' ')
print()

C:\Users\james\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in longlong_scalars


IndexError: list index out of range

In [13]:
#KD3

p = Path(init)
c=0
for x in p.iterdir():
    
    try:
        image16 = cv2.imread(str(x), cv2.IMREAD_GRAYSCALE)

        if(image16.shape!=(41,41)):
            print('yes')

        img8 = image16

        all_zeros = not np.any(img8)
        if(all_zeros):
            continue

        coeffs2 = pywt.dwt2(img8, 'db1')
        LL, (LH, HL, HH) = coeffs2

        width = 41
        height = 41
        dim = (width, height)

        # resize image
        LL = (cv2.resize(LL, dim, interpolation = cv2.INTER_AREA)).astype('uint8')
        LH = (cv2.resize(LH, dim, interpolation = cv2.INTER_AREA)).astype('uint8')
        HL = (cv2.resize(HL, dim, interpolation = cv2.INTER_AREA)).astype('uint8')
        HH = (cv2.resize(HH, dim, interpolation = cv2.INTER_AREA)).astype('uint8')

        #esimg = esi(img8)


        w = str(x)
        y = os.path.normpath(KD3+ '//' + w[w.rfind('\\')+1: w.rfind('.')])

        # Writing Files
        cv2.imwrite(str(y)+'-orig.png',img8)
        cv2.imwrite(str(y)+'-LL.png',LL)
        cv2.imwrite(str(y)+'-LH.png',LH)
        cv2.imwrite(str(y)+'-HL.png',HL)
        cv2.imwrite(str(y)+'-HH.png',HH)
        #cv2.imwrite(str(y)+'-esi.png',esimg)
        #print(gr) #  Run check

        c = c+1
    except:
        continue

print('Total FILES Processed : ', c, end=' ')
print()


Total FILES Processed :  913 


In [14]:
#KD1
p = Path(init)
c=0
for x in p.iterdir():
    
    try:
        image16 = cv2.imread(str(x), cv2.IMREAD_GRAYSCALE)

        if(image16.shape!=(41,41)):
            print('yes')

        img8 = image16

        all_zeros = not np.any(img8)
        if(all_zeros):
            continue
            
        

        
        gauss3 = (ndimage.gaussian_filter(img8, 1, 3, mode='wrap')).astype('uint8')
        gauss5 = (ndimage.gaussian_filter(img8, 1, 5, mode='wrap')).astype('uint8')
        #esimg = esi(img8)


        w = str(x)
        y = os.path.normpath(KD1+ '//' + w[w.rfind('\\')+1: w.rfind('.')])

        # Writing Files
        cv2.imwrite(str(y)+'-orig.png',img8)
        cv2.imwrite(str(y)+'-g3.png',gauss3)
        cv2.imwrite(str(y)+'-g5.png',gauss5)
        #cv2.imwrite(str(y)+'-esi.png',esimg)
        #print(gr) #  Run check
        classset.add(int((str(x).split("\\")[-1])[6:9]))

        c = c+1
    except:
        continue


In [16]:
#KD2
p = Path(init)
c=0
for x in p.iterdir():
    
    try:
        image16 = cv2.imread(str(x), cv2.IMREAD_GRAYSCALE)

        if(image16.shape!=(41,41)):
            print('yes')

        img8 = image16

        all_zeros = not np.any(img8)
        if(all_zeros):
            continue
            
        

        
        gauss3 = (ndimage.gaussian_filter(img8, 1, 3, mode='wrap')).astype('uint8')
        gauss5 = (ndimage.gaussian_filter(img8, 1, 5, mode='wrap')).astype('uint8')
        esimg = esi(img8)


        w = str(x)
        y = os.path.normpath(KD2+ '//' + w[w.rfind('\\')+1: w.rfind('.')])

        # Writing Files
        cv2.imwrite(str(y)+'-orig.png',img8)
        cv2.imwrite(str(y)+'-g3.png',gauss3)
        cv2.imwrite(str(y)+'-g5.png',gauss5)
        cv2.imwrite(str(y)+'-esi.png',esimg)
        #print(gr) #  Run check
        classset.add(int((str(x).split("\\")[-1])[6:9]))

        c = c+1
    except:
        continue


C:\Users\james\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\james\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in longlong_scalars
